In [ ]:
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import statistics
import h5py
import zipfile
import gc
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [ ]:
# Accessing Data to train our deep learning Model
with h5py.File("Dataset.h5", 'r') as hf:
    imv_feature = hf['image_vector'][:]
    glm_target = hf['glomeruli_type'][:]

In [ ]:
# Stratified split to ensure balanced classes in both sets
x_train, x_test, y_train, y_test = train_test_split(combined_final_im_vector, combined_final_glomeruli, test_size=0.3, stratify=combined_final_glomeruli, random_state=35)

In [ ]:
# Input layer to process the image vectors
input_layer = Input(shape=(300, 300, 3))

In [ ]:
# Load the EfficientNetB3 model with no pre-trained weights. 
base_model = EfficientNetB3(include_top=False, input_tensor=input_layer, weights="imagenet")

In [ ]:
# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Fit the model
batches = 32
total_epochs = 10

In [ ]:
# Train the model
b3ic = model.fit(
    # datagen.flow(x_train, y_train, batch_size=batch_size),
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=total_epochs,
    batch_size = batches
    )

In [ ]:
model.save('b3ic.keras')

In [ ]:
# Unfreeze some layers and fine-tune the model
for layer in base_model.layers:
    layer.trainable = True
# Compile the model again
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Fine-tune the model
fine_tune_batches = 64
fine_tune_epochs = 20

In [ ]:
# Train the Model Again
fine_tuned_b3ic = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=fine_tune_epochs,
    batch_size = fine_tune_batches
)

In [ ]:
# Final Model is saved as "final-b3ic.keras"
model.save('final-b3ic.keras')

In [ ]:
# Make predictions on the test set
y_pred = model.predict(x_test)

In [ ]:
# Calculate precision and recall
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print('Confusion Matrix:\n', conf_matrix)